<a href="https://colab.research.google.com/github/jo1132/Yolo5_objectdetection_golf/blob/master/Yolo5_Tutorial_Golf_Miniset_setting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라벨링 데이터 CSV 만들기

In [143]:
import os             # 파일 읽어오기, 리스트 불러오기
import shutil         # 파일 복사하기
import json           # JSON 읽기
import pandas as pd   # DataFrame 만들기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 폴더안에 모든 Json파일경로 리스트로 가져오기

In [ ]:
# 시작 폴더 경로
Base_Path = '/content/drive/MyDrive/Colab Notebooks/공부용/golf/2020-02-097.스포츠사람동작(골프)_sample'
total_path_list = []

def RecursiveJsonFinder(PATH):
    file_list = os.listdir(PATH)
    
    for file in file_list:
        file_path = PATH+'/'+file
        if(os.path.isdir(file_path)):
            RecursiveJsonFinder(file_path)
        elif (os.path.splitext(file_path)[-1] == '.json'):
            total_path_list.append(file_path)
RecursiveJsonFinder(Base_Path)
print(len(total_path_list))

1709


## Json파일경로로 Json정보 DataFrame만들고 저장하기

### Categories 정리

In [ ]:
def DF_categories(data): # return Categories DataFrame
    list_to_string = (', ').join(data['keypoints'])
    df = pd.DataFrame(list_to_string, columns=['keypoints'], index=[0])
    return df

### image정보 정리

In [ ]:
def DF_image(data): # return Image DataFrame
    df = pd.DataFrame([data['filename'], data['action'], data['evaluation'], data['hitting']], index=['filename', 'action', 'evaluation', 'hitting']).T
    return df

### actor 정보 정리

In [ ]:
def DF_actor(data): # return Actor DataFrame
    return pd.DataFrame(data, index=[0])

### Anntations 정보 정리

In [ ]:
def DF_annotations(data): # return Categories DataFrame
    class_list = {'person':0, 'ball':1, 'club':2}
    column_list = []
    for cls in class_list.keys():
          column_list.append(cls) 
    total_list = [[] for i in range(3)]
    
    for item in data:
        class_idx = class_list[item['class']]
        bucket = item.get('box', 0)
        if(bucket):
            total_list[class_idx].append(bucket)

    df = pd.DataFrame(data=total_list, index=column_list).T
    return df

### 각 정보를 정리한 DataFrame 하나로 붙혀주기

In [ ]:
def GetOneLineDF(PATH):
    data = json.load(open(PATH, 'rt', encoding='UTF=8'))
    df_annotations = DF_annotations(data['annotations'])
    df_categories = DF_categories(data['categories'])
    df_image = DF_image(data['image'])
    df_actor = DF_actor(data['actor'])
    
    return pd.concat([df_categories, df_image, df_actor, df_annotations], axis=1)

## total-path_list에서 하나씩 꺼내서 Json정보를 DataFrame으로 만들고, CSV저장

In [ ]:
import pandas as pd
import json
import os
pd.options.display.max_colwidth = 200

total_df = []

for count, PATH in enumerate(total_path_list):

    if len(total_df):
        total_df = pd.concat([total_df, GetOneLineDF(PATH)], ignore_index=True)
    else:
        total_df = GetOneLineDF(PATH)


image_path_list = []

for PATH in total_path_list:
    split_PATH = PATH.split('/')
    split_PATH[8] = '원천데이터'
    PATH = ('/').join(split_PATH)
    PATH = PATH.replace('.json', '.jpg')
    image_path_list.append(PATH)

image_df = pd.DataFrame(image_path_list, columns=['image_path'])
total_df = pd.concat([image_df, total_df], axis=1)
total_df = total_df.dropna()
total_df.to_csv(Base_Path+'/total_Swing.csv', index=False, encoding="utf-8-sig")



# MiniSet 만들기

## box 데이터 label로 만들기

In [153]:
import pandas as pd
import shutil

# box데이터를 YOLO좌표료 변환
def str_to_list(data, crop, image_size):
    '''
        xmin: top-left x coordinate,
        ymin: top-left y coordinate,
        w: bounding box width,
        h: bounding box height,
        w_img: image width,
        h_img: image height
    '''
    try:
      x, y, w, h = map(int, data[1:-1].split(', '))
      x -= crop
      xcenter = (x + w/2) / image_size
      ycenter = (y + h/2) / image_size
      w = w / image_size
      h = h / image_size
      if xcenter < 0 or xcenter > 1:
        return [0]
      return [xcenter, ycenter, w, h]
    except:
      return [0]

def Build_label(total_df, save_path):
  boxes = []
  data = ''
  for idx, col in enumerate(['person', 'club', 'ball']):
    line = (' ').join(list(map(str, [idx] + str_to_list(total_df[col], (1920-1080)/2, 1080))))
    if(line):
      data += line + '\n'
  if(len(data)>3):
    file_name = total_df['filename'].replace('.jpg', '.txt')
    with open(save_path+'/labels/'+file_name, 'w') as f:
      f.write(data)
      f.close
  else:
    return False
  return True

#Build_label(total_df, label_path)

## image 편집하기

In [154]:
from PIL import Image

def Image_Crop_and_Resize(total_df, bef, aft, save_path):
  x = (bef - aft)/2
  y = 0
  w = aft
  h = aft

  img = Image.open(total_df['image_path'])
  croped_img = img.crop((x, y, x+w, y+h))
  resized_img = croped_img.resize((512,512))
  resized_img.save(save_path+'/images/'+total_df['image_path'].split('/')[-1])

## 메인

In [155]:
def Build_Miniset(total_df, save_path):
  for idx in range(len(total_df)):
    if Build_label(total_df.iloc[idx], save_path):
      try:
        Image_Crop_and_Resize(total_df.iloc[idx], 1920,1080, save_path)
      except:
        print(total_df['image_path'].iloc[idx])

## 실행

In [156]:
import pandas as pd

total_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/공부용/golf/2020-02-097.스포츠사람동작(골프)_sample/total_Swing.csv')
save_path = '/content/drive/MyDrive/Colab Notebooks/공부용/golf/dataset'
Build_Miniset(total_df, save_path)

/content/drive/MyDrive/Colab Notebooks/공부용/golf/2020-02-097.스포츠사람동작(골프)_sample/원천데이터/Association/Female/Putting/20200517_General_012_DOC_P_F20_MM/20200517_General_012_DOC_P_F20_MM_015_0002.jpg
/content/drive/MyDrive/Colab Notebooks/공부용/golf/2020-02-097.스포츠사람동작(골프)_sample/원천데이터/Association/Female/Putting/20200517_General_012_DOC_P_F20_MM/20200517_General_012_DOC_P_F20_MM_015_0202.jpg
/content/drive/MyDrive/Colab Notebooks/공부용/golf/2020-02-097.스포츠사람동작(골프)_sample/원천데이터/Association/Female/Putting/20200517_General_012_DOC_P_F20_MM/20200517_General_012_DOC_P_F20_MM_015_0602.jpg
/content/drive/MyDrive/Colab Notebooks/공부용/golf/2020-02-097.스포츠사람동작(골프)_sample/원천데이터/Association/Female/Putting/20200517_General_012_DOC_P_F20_MM/20200517_General_012_DOC_P_F20_MM_015_0802.jpg
/content/drive/MyDrive/Colab Notebooks/공부용/golf/2020-02-097.스포츠사람동작(골프)_sample/원천데이터/Association/Female/Putting/20200517_General_012_DOC_P

# 테스트

In [ ]:
import pandas as pd
total_df = pd.read_csv(Base_Path+'/total_Swing.csv')
total_df = total_df.fillna('0')
image_list = total_df['filename'].tolist()
image_path = total_df['image_path'].tolist()
copy_path = '/content/drive/MyDrive/Colab Notebooks/공부용/golf'+'/'+miniset_name+'/images'

ERR_list = []

for idx in range(len(total_df)):
    image_name = total_df.iloc[idx].filename
    image_path = total_df.iloc[idx].image_path
    person = total_df.iloc[idx].person
    club = total_df.iloc[idx].club
    ball = total_df.iloc[idx].ball

    try:
        #!cp  image_path copy_path+'/'+image_name
        shutil.copyfile(image_path, copy_path+'/'+image_name)
    except:
        ERR_list.append(image_path)